# 서울시 구별 주유소 데이터 분석 및 시각화

In [14]:
import numpy as np
import pandas as pd
import requests
from urllib.parse import quote
import folium, json

In [3]:
df = pd.read_csv(r'C:/workspace/서울_주유소(230418).csv')

In [6]:
# 구별 집계 분석
gu_mm = df.groupby('구')['가격'].agg(['mean', 'min', 'max'])
gu_mm

,mean,min,max
구,,,
강남구,1942.294118,1623,2578
강동구,1735.000000,1669,1978
강북구,1649.916667,1629,1707
강서구,1696.818182,1628,1995
관악구,1744.714286,1685,1852
광진구,1684.538462,1654,1748
구로구,1737.200000,1645,2198
금천구,1699.090909,1665,1769
노원구,1699.428571,1629,1795


In [7]:
# 해당구에서 최대값 최소값만 구하기
# 예 강남구
_, mi, ma = gu_mm.loc['강남구']
mi, ma = int(mi), int(ma)
mi, ma

(1623, 2578)

In [8]:
# 컬럼을 하나 추가하여 최대값인지 최속값인지 결정해서 넣을 예정
df['mm'] = np.zeros(df.shape[0], int)
df.head()

,상호,주소,가격,셀프,상표,구,mm
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,0
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),1669,Y,SK에너지,강남구,0
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),1684,Y,현대오일뱅크,강남구,0
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),1687,Y,GS칼텍스,강남구,0
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1704,Y,SK에너지,강남구,0


In [17]:
# mm 컬럼에 최소값이 같으면 1을 넣고 최대값과 같으면 2를 넣을것
for gu in gu_mm.index:
    _, mi, ma = gu_mm.loc[gu]
    mi, ma = int(mi) , int(ma)
    for i in df.index:
        if df.가격[i] == mi and df.구[i] == gu:
            df.mm[i] = 1
        if df.가격[i] == ma and df.구[i] == gu:
            df.mm[i] = 2

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_10392\1260248008.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mm[i] = 1
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_10392\1260248008.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mm[i] = 2


In [18]:
# mm 컬럼에 데이터가 있는 행만 추출
df1 = df[df.mm !=0]
df1.head()

,상호,주소,가격,셀프,상표,구,mm
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,1
33,(주)만정에너지 삼보주유소,서울 강남구 봉은사로 433 (삼성동),2578,N,GS칼텍스,강남구,2
34,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1669,Y,현대오일뱅크,강동구,1
46,광성주유소,서울 강동구 올림픽로 673 (천호동),1978,N,S-OIL,강동구,2
47,SK에너지(주) 매일주유소,서울 강북구 삼양로 217 (미아동),1629,Y,SK에너지,강북구,1


In [19]:
df1.shape

(70, 7)

# 지도 시각화

In [20]:
# KAKAO API로 위,경도값을 추출
with open('C:/workspace/data1/kakaoAPI.txt') as file:
    kakao_key = file.read()

In [26]:
lat_list = [] 
lng_list = []
for i in df1.index:
    search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    url = f'{search_url}?query={quote(df1.주소[i])}'
    result = requests.get(url, headers={"Authorization" : f'KakaoAK {kakao_key}'}).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df1.상호[i])

In [27]:
df1['위도'] = lat_list
df1['경도'] = lng_list
df1.head()

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_10392\2420660999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['위도'] = lat_list
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_10392\2420660999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['경도'] = lng_list


,상호,주소,가격,셀프,상표,구,mm,위도,경도
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,1,37.467272,127.118743
33,(주)만정에너지 삼보주유소,서울 강남구 봉은사로 433 (삼성동),2578,N,GS칼텍스,강남구,2,37.511551,127.047115
34,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1669,Y,현대오일뱅크,강동구,1,37.536417,127.149372
46,광성주유소,서울 강동구 올림픽로 673 (천호동),1978,N,S-OIL,강동구,2,37.541577,127.124242
47,SK에너지(주) 매일주유소,서울 강북구 삼양로 217 (미아동),1629,Y,SK에너지,강북구,1,37.622765,127.019086


In [30]:
filename = './data1/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

In [34]:
map = folium.Map(location=[df1.위도.mean(), df1.경도.mean()], zoom_start=12)
for i in df1.index:
    folium.Marker(
        location=[df1.위도[i], df1.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=f'{df1.상호[i]}<br>휘발유:{df1.가격[i]:,d}원',
        icon=folium.Icon(color='green' if df.mm[i] == 1 else 'red',
                        icon = 'glyphicon glyphicon-thumbs-up' 
                         if df.mm[i] == 1 else 'glyphicon glyphicon-thumbs-down')
    ).add_to(map)
map

In [ ]:
# 단계구분도 나중에...과제 평균으로 단계 구분도를 그리고 주유소 .
# stamen Toner